### RTX A6000 测试：Qwen/Qwen3-1.7B

In [ ]:
!pip install uv
!mkdir sglang
!cd sglang
!uv venv
!source .venv/bin/activate
!uv pip install "sglang[all]>=0.4.9.post3"
!apt-get update && apt-get install -y libnuma-dev
!pip install --upgrade "filelock>=3.13.1"
!cd ..
!mkdir vllm
!cd vllm
!uv venv
!source .venv/bin/activate
!uv pip install "vllm>=0.10.0"

In [ ]:
!python -m sglang.launch_server --model-path Qwen/Qwen3-1.7B

In [ ]:
!cd sglang
!source .venv/bin/activate
!python -m sglang.bench_serving --backend sglang --model Qwen/Qwen3-1.7B --dataset-name random --num-prompts 1000 --random-input-len 256 --random-output-len 100 --random-range-ratio 1 --profile

```
============ Serving Benchmark Result ============
Backend:                                 sglang    
Traffic request rate:                    inf       
Max request concurrency:                 not set   
Successful requests:                     1000      
Benchmark duration (s):                  175.19    
Total input tokens:                      256000    
Total generated tokens:                  100000    
Total generated tokens (retokenized):    99939     
Request throughput (req/s):              5.71      
Input token throughput (tok/s):          1461.26   
Output token throughput (tok/s):         570.81    
Total token throughput (tok/s):          2032.07   
Concurrency:                             97.17     
----------------End-to-End Latency----------------
Mean E2E Latency (ms):                   17023.71  
Median E2E Latency (ms):                 16988.58  
---------------Time to First Token----------------
Mean TTFT (ms):                          4409.82   
Median TTFT (ms):                        4246.55   
P99 TTFT (ms):                           7725.49   
---------------Inter-Token Latency----------------
Mean ITL (ms):                           127.45    
Median ITL (ms):                         83.95     
P95 ITL (ms):                            208.67    
P99 ITL (ms):                            788.18    
Max ITL (ms):                            6627.63   
==================================================
```

In [ ]:
!vllm serve Qwen/Qwen3-1.7B

In [ ]:
!cd vllm
!source .venv/bin/activate
!vllm bench serve --model Qwen/Qwen3-1.7B --num-prompts 1000 --random-input-len 256 --random-output-len 100 --ignore-eos

```
============ Serving Benchmark Result ============
Successful requests:                     1000      
Benchmark duration (s):                  21.16     
Total input tokens:                      255191    
Total generated tokens:                  100000    
Request throughput (req/s):              47.26     
Output token throughput (tok/s):         4725.63   
Total Token throughput (tok/s):          16785.01  
---------------Time to First Token----------------
Mean TTFT (ms):                          9488.78   
Median TTFT (ms):                        8711.83   
P99 TTFT (ms):                           18559.76  
-----Time per Output Token (excl. 1st token)------
Mean TPOT (ms):                          47.13     
Median TPOT (ms):                        50.22     
P99 TPOT (ms):                           53.97     
---------------Inter-token Latency----------------
Mean ITL (ms):                           47.13     
Median ITL (ms):                         28.15     
P99 ITL (ms):                            93.13     
==================================================
```

### 4 * H100 SXM 测试：DeepSeek-V2.5-1210-FP8

In [ ]:
!huggingface-cli download RedHatAI/DeepSeek-V2.5-1210-FP8 --local-dir deepseek --local-dir-use-symlinks False

In [ ]:
!pip install uv
!mkdir sglang
!cd sglang
!uv venv
!source .venv/bin/activate
!uv pip install "sglang[all]>=0.4.9.post3"
!apt-get update && apt-get install -y libnuma-dev
!pip install --upgrade "filelock>=3.13.1"
!cd ..
!mkdir vllm
!cd vllm
!uv venv
!source .venv/bin/activate
!uv pip install "vllm>=0.10.0"

In [ ]:
!python -m sglang.launch_server --model deepseek --tp 4 --enable-ep-moe

In [ ]:
!cd sglang
!source .venv/bin/activate
!python -m sglang.bench_serving --backend sglang --model deepseek --dataset-name random --num-prompts 1000 --random-input-len 256 --random-output-len 100 --random-range-ratio 1 --profile

In [ ]:
!export VLLM_USE_DEEP_GEMM=1 
!export VLLM_ALL2ALL_BACKEND="deepep_high_throughput"
!export VLLM_RANDOMIZE_DP_DUMMY_INPUTS=1
!vllm serve deepseek --enforce-eager --enable-expert-parallel -tp 8

In [ ]:
!cd vllm
!source .venv/bin/activate
!vllm bench serve --model deepseek --num-prompts 1000 --random-input-len 256 --random-output-len 100 --ignore-eos